In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from import_file import  *
load_libs()

# 最低级别段的计算
## 提取高低点分形
### 读取k线数据

In [3]:
instrument = Instrument()
code = 'il9'
df = instrument.bar(code, asset='future')
hl_df = df.loc[:, ['high', 'low']]
hl_df['kindex'] = np.arange(len(hl_df))

### 寻找k线高点

In [250]:
HH = pd.DataFrame()
#HH['-2'] = hl_df.high.shift(2)
HH['-1'] = hl_df.high.shift(1)
HH['0'] = hl_df.high
HH['1'] = hl_df.high.shift(-1)
#HH['2'] = hl_df.high.shift(-2)
HH = hl_df.loc[:, ['high', 'kindex']].loc[HH.idxmax(axis=1) == '0']
HH.columns = ['peak', 'kindex']

### 寻找k线低点

In [251]:
LL = pd.DataFrame()
#LL['-2'] = hl_df.low.shift(2)
LL['-1'] = hl_df.low.shift(1)
LL['0'] = hl_df.low
LL['1'] = hl_df.low.shift(-1)
#LL['2'] = hl_df.low.shift(-2)
LL = hl_df.loc[:, ['low', 'kindex']].loc[LL.idxmin(axis=1) == '0']
LL.columns = ['peak', 'kindex']
LL.peak = -LL.peak

### 获取高低分形DF

In [286]:
gdfx = pd.concat([HH, LL])
gdfx = gdfx.sort_index()

### 去除连续高点或者低点
    高点不能比低点低，低点不能比高点高

In [288]:
# flag = pd.Series([False] * 3)
# segment = gdfx
# while not flag.all():
#     t_peak = segment.peak
#     t_peak_m1 = segment.peak.shift(1)
#     t_peak_p1 = segment.peak.shift(-1)
#     t_sign = t_peak / abs(t_peak)
#     t_peak_m1.iat[0] = t_peak.iat[0] - 1
#     t_peak_p1.iat[-1] = t_peak.iat[-1] - 1
#     yflag = abs((t_peak / t_peak_m1) ** t_sign) >= 1
#     tflag = abs((t_peak / t_peak_p1) ** t_sign) >= 1

#     flag = np.logical_and(yflag, tflag)
#     segment = segment[flag]

bi = removed_continue_peak(gdfx)
bi.iloc[[0,-1]]

,peak,kindex
datetime,,
2013-10-18,984.0,0
2017-11-02,455.0,988


### 去除相邻较近的高低点

In [289]:
# num = 2
# t_index = segment.kindex
# t_index_m1 = segment.kindex.shift(1)
# t_index_m1.iat[0] = t_index.iat[0] - num -1
# nflag = t_index - t_index_m1 > num

# t_peak = segment.peak
# t_peak_m2 = segment.peak.shift(2)
# t_peak_p2 = segment.peak.shift(-2)
# t_sign = t_peak / abs(t_peak)
# t_peak_m2.iat[0] = t_peak.iat[0] - 1
# t_peak_m2.iat[1] = t_peak.iat[1] - 1
# t_peak_p2.iat[-1] = t_peak.iat[-1] - 1
# t_peak_p2.iat[-2] = t_peak.iat[-2] - 1
# yflag = abs((t_peak / t_peak_m2) ** t_sign) > 1
# tflag = abs((t_peak / t_peak_p2) ** t_sign) > 1

# flag = np.logical_or(nflag, np.logical_and(yflag, tflag))

# Nflag = -(segment.peak.shift(1) / segment.peak.shift(-1)) ** t_sign > 1  # 删除点后，前面的点要比后面的点高或者低
# flag = np.logical_or(Nflag, flag)

# upsegment = segment[flag]
segment = removed_near_peak(bi, 0)

len(upsegment)

G:\Code\Crawler\ts_zcsz.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  def __init__(self, ks):


68

In [255]:
def segment_revised(segment, upsegment):
    x = segment.join(upsegment,lsuffix='_x', rsuffix='_y')
    x.fillna(method='bfill', inplace=True)
    x['diff'] = abs((x['peak_y'] / x['peak_x']) ** (x['peak_y'] / abs(x['peak_y']))) <= 1
    up_segment = segment[x['diff']]

    up_segment = removed_continue_peak(up_segment)

    x = segment.join(up_segment, lsuffix='_x', rsuffix='_y')
    x.fillna(method='ffill', inplace=True)
    x['diff'] = abs((x['peak_y'] / x['peak_x']) ** (x['peak_y'] / abs(x['peak_y']))) <= 1
    up_segment = segment[x['diff']]
    
    up_segment = removed_continue_peak(up_segment)
    return up_segment

In [290]:
segment = segment_revised(bi, segment)
len(segment)

313

### 段升级，段的端点必须是区域内的极值点

In [313]:
def upgrade_segment(segment):
    upsegment = pd.DataFrame()
    if len(segment) < 7:
        return segment
    t_peak = segment.peak
    t_peak_m2 = segment.peak.shift(2)
    t_peak_p2 = segment.peak.shift(-2)
    t_sign = t_peak / abs(t_peak)
    t_peak_m2.iat[0] = t_peak.iat[0] - 1
    t_peak_m2.iat[1] = t_peak.iat[1] - 1
    t_peak_p2.iat[-1] = t_peak.iat[-1] - 1
    t_peak_p2.iat[-2] = t_peak.iat[-2] - 1
    yflag = abs((t_peak / t_peak_m2) ** t_sign) > 1
    tflag = abs((t_peak / t_peak_p2) ** t_sign) >= 1
    flag = np.logical_and(yflag, tflag)
    upsegment = segment[flag]
    upsegment = removed_continue_peak(upsegment)
    upsegment = segment_revised(segment, upsegment)
    return upsegment

In [314]:
upsegment = upgrade_segment(segment)
upsegment.iloc[[0,-1]]
len(upsegment)

,peak,kindex
datetime,,
2013-10-18,984.0,0
2017-10-30,-427.5,985


60

In [315]:
segment_verify(segment, upsegment).any()

False

In [317]:
chart

## 调用函数分段

In [4]:
instrument = Instrument()
code = 'il9'
df = instrument.bar(code, asset='future')
gdfx = calculate_gdfx(df)
segment = calculate_segment(gdfx)
segments = [segment]

In [6]:
df.head()

,open,high,low,close,openInt,volume,comment
datetime,,,,,,,
2013-10-18,974.0,984.0,962.0,976.0,78656,338700,0
2013-10-21,974.0,976.0,959.0,967.0,82136,104026,0
2013-10-22,965.0,965.0,946.0,946.0,89528,98812,0
2013-10-23,948.0,951.0,935.0,938.0,94634,122456,0
2013-10-24,937.0,937.0,916.0,925.0,103770,177400,0


In [5]:
while True:
    upsegment = upgrade_segment(segments[-1])
    if upsegment.empty:
        break
    segments.append(upsegment)

## 图形化
### 日线分段

In [189]:
chart = Highstock(width=1100, height=600)
chart.JSsource = [
    'Api/static/jquery/jquery-3.2.1.min.js',
    'Api/static/highcharts/highstock.js',
    'Api/static/highcharts/modules/exporting.js',
    'Api/static/highcharts/css/highcharts-more.js'
]
options = {
        'rangeSelector': {
            'selected': 1,
            'inputDateFormat': '%Y-%m-%d'
        },
        'title': {
            'text': code
        },
        'xAxis': {
            'dateTimeLabelFormats': {
                'millisecond': '%H:%M:%S.%L',
                'second': '%H:%M:%S',
                'minute': '%H:%M',
                'hour': '%H:%M',
                'day': '%m-%d',
                'week': '%m-%d',
                'month': '%y-%m',
                'year': '%Y'
            }
        },
        'yAxis': [{
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '股价'
            },
            'height': '60%',
            'lineWidth': 2
        }, {
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '成交量'
            },
            'top': '65%',
            'height': '35%',
            'offset': 0,
            'lineWidth': 2
        }],
        'legend':{
            'enabled': True,
            'layout': 'vertical',
            'align': 'right',
            'verticalAlign': 'middle',
            'borderWidth': 0
        },
}

chart.set_dict_options(options)

In [190]:
df_draw = df.reset_index()
ohlc = df_draw.loc[:, ['datetime', 'open', 'high', 'low', 'close']]
ohlc = ohlc.values.tolist()
volume = df_draw.loc[:, ['datetime','volume']]
volume = volume.values.tolist()
openInt = df_draw.loc[:, ['datetime','openInt']]
openInt = openInt.values.tolist()
chart.add_data_set(ohlc, series_type='candlestick', name=code, color='green', lineColor='green', upColor='red', upLineColor='red')
chart.add_data_set(volume, series_type='column', name='volume', yAxis=1)
chart.add_data_set(openInt , name='openInt', yAxis=1)

In [191]:
for i,segment in enumerate(segments):
    segment_draw = abs(segment).reset_index()
    segment_peak = segment_draw.loc[:, ['datetime','peak']]
    segment_peak = segment_peak.values.tolist()
    chart.add_data_set(segment_peak , name='segment{}'.format(i), lineWidth=1+i*2)

In [192]:
chart

In [205]:
if len(segments) > 1:
    df = segments[1]
else:
    df = segments[-1]
start_date = df.index[-2]    
start_date

Timestamp('2017-10-12 00:00:00')

In [206]:
instrument = Instrument()
code = 'il9'
df = instrument.bar(code, asset='future', start_date=start_date, ktype='1min')
gdfx = calculate_gdfx(df)
segment = calculate_segment(gdfx)
segments = [segment]

In [207]:
while True:
    upsegment = upgrade_segment(segments[-1])
    if upsegment.empty:
        break
    segments.append(upsegment)

In [208]:
chart_min = Highstock(width=1100, height=600)
chart_min.JSsource = [
    'Api/static/jquery/jquery-3.2.1.min.js',
    'Api/static/highcharts/highstock.js',
    'Api/static/highcharts/modules/exporting.js',
    'Api/static/highcharts/css/highcharts-more.js'
]
options = {
        'title': {
            'text': code
        },
        'rangeSelector' : {
                'buttons': [{
                    'type': 'hour',
                    'count': 1,
                    'text': '1h'
                }, {
                    'type': 'day',
                    'count': 1,
                    'text': '1d'
                }, {
                    'type': 'month',
                    'count': 1,
                    'text': '1m'
                }, {
                    'type': 'year',
                    'count': 1,
                    'text': '1y'
                }, {
                    'type': 'all',
                    'text': 'All'
                }],
                'inputEnabled': False, # it supports only days
                'selected' : 2 #/ all
            },
        'xAxis': {
            'dateTimeLabelFormats': {
                'millisecond': '%H:%M:%S.%L',
                'second': '%H:%M:%S',
                'minute': '%H:%M',
                'hour': '%H:%M',
                'day': '%m-%d',
                'week': '%m-%d',
                'month': '%y-%m',
                'year': '%Y'
            },
            'minRange': 3600 * 1000
        },
        'yAxis': [{
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '股价'
            },
            'height': '70%',
            'lineWidth': 2
        }, {
            'labels': {
                'align': 'right',
                'x': -3
            },
            'title': {
                'text': '成交量'
            },
            'top': '75%',
            'height': '25%',
            'offset': 0,
            'lineWidth': 2
        }],
        'legend':{
            'enabled': True,
            'layout': 'vertical',
            'align': 'right',
            'verticalAlign': 'middle',
            'borderWidth': 0
        },
}

chart_min.set_dict_options(options)

In [209]:
df_draw = df.reset_index()
ohlc = df_draw.loc[:, ['index', 'open', 'high', 'low', 'close']]
ohlc = ohlc.values.tolist()
volume = df_draw.loc[:, ['index','volume']]
volume = volume.values.tolist()
# openInt = df_draw.loc[:, ['index','openInt']]
# openInt = openInt.values.tolist()
chart_min.add_data_set(ohlc, series_type='candlestick', name=code, color='green', lineColor='green', upColor='red', upLineColor='red')
chart_min.add_data_set(volume, series_type='column', name='volume', yAxis=1)
# chart_min.add_data_set(openInt , name='openInt', yAxis=1)

In [210]:
for i,segment in enumerate(segments):
    segment_draw = abs(segment).reset_index()
    segment_peak = segment_draw.loc[:, ['index','peak']]
    segment_peak = segment_peak.values.tolist()
    chart_min.add_data_set(segment_peak , name='segment{}'.format(i), lineWidth=1+i*2)

In [ ]:
chart_min

In [161]:
segment_peak

[[Timestamp('2017-08-21 13:45:00'), 4460.0],
 [Timestamp('2017-09-21 10:35:00'), 3899.0],
 [Timestamp('2017-10-20 10:41:00'), 4265.0],
 [Timestamp('2017-10-31 09:03:00'), 4082.0]]